In [1]:
import numpy as np
import pandas as pd
from textwrap import wrap
from matplotlib import pyplot as plt
from sklearn import metrics
import matplotlib.ticker as mticker
import sys
from hsbmpy import plot_topic_size, get_max_available_L

In [2]:
directory="/home/fvalle/phd/datasets/tcga/TCGA/"

In [3]:
algorithm = 'lda'
L = get_max_available_L(directory, algorithm=algorithm)

# topic size

In [ ]:
for l in range(0,L+1):
    plot_topic_size(directory,l, algorithm=algorithm)

# Geneontology

In [4]:
from geneontology import get_ontology_df, ensg_to_symbol
from tableanalyser import get_symbol

In [5]:
l=L
df_topics = pd.read_csv("%s/%s/%s_level_%d_topics.csv"%(directory,algorithm,algorithm,l))

In [6]:
#import gseapy as gs
#gs.get_library_name()

In [7]:
back_sea=[]
for topic in df_topics.columns:
    for g in df_topics[topic].dropna().values:
        try:
            back_sea.append(get_symbol(g))
        except:
            print("error %s"%g)

In [8]:
for itopic,topic in enumerate(df_topics.columns):
    try:
        enriched_topic = pd.read_csv("%s/%s/gsea/gsea_level_%d_topic%d.csv"%(directory,algorithm,l,itopic), index_col=[0])
        print(topic)
    except:
        try:
            symbols = ensg_to_symbol(df_topics.loc[:,topic].dropna().values)
            print(topic)
            enriched_topic = get_ontology_df(symbols, background=back_sea).sort_values(by=['Adjusted P-value'], ascending=True)
            enriched_topic = enriched_topic.loc[enriched_topic.index.values[:20],:]
            enriched_topic.to_csv("%s/%s/gsea/gsea_level_%d_topic%d.csv"%(directory,algorithm,l,itopic))
        except:
            print(*sys.exc_info())
    print(enriched_topic)

Topic 1


2019-12-16 16:27:31,696 Warning: No enrich terms using library GO_Molecular_Function_2018 when cutoff = 0.05
2019-12-16 16:27:39,738 Warning: No enrich terms using library Human_Phenotype_Ontology when cutoff = 0.05
2019-12-16 16:27:48,307 Warning: No enrich terms using library Tissue_Protein_Expression_from_Human_Proteome_Map when cutoff = 0.05
2019-12-16 16:27:53,381 Warning: No enrich terms using library NCI-60_Cancer_Cell_Lines when cutoff = 0.05


                                                   Term  Adjusted P-value  \
292                     cytosolic ribosome (GO:0022626)          0.000074   
71                 protein targeting to ER (GO:0045047)          0.000083   
74                     viral transcription (GO:0019083)          0.000089   
70    cotranslational protein targeting to membrane ...          0.000101   
73    nuclear-transcribed mRNA catabolic process, no...          0.000102   
72                   viral gene expression (GO:0019080)          0.000116   
293                         cytosolic part (GO:0044445)          0.000126   
3611                                           Ribosome          0.000139   
69    SRP-dependent cotranslational protein targetin...          0.000162   
294                         focal adhesion (GO:0005925)          0.000199   
374    GTEX-XYKS-1726-SM-4E3IO_ovary_female_60-69_years          0.000379   
76            peptide biosynthetic process (GO:0043043)          0.000561   

2019-12-16 16:28:12,848 Warning: No enrich terms using library Tissue_Protein_Expression_from_Human_Proteome_Map when cutoff = 0.05


                                                   Term  Adjusted P-value  \
44       extracellular matrix organization (GO:0030198)      1.334870e-21   
2817                           ECM-receptor interaction      1.809538e-14   
570     GTEX-Q734-0008-SM-48U2I_skin_female_40-49_years      5.574095e-12   
571   GTEX-RWS6-0426-SM-47JXH_blood vessel_female_60...      7.031393e-12   
569     GTEX-XMD3-0008-SM-4AT4V_skin_female_50-59_years      1.013217e-11   
572     GTEX-W5X1-0008-SM-4LMKA_skin_female_40-49_years      1.810507e-11   
298            endoplasmic reticulum lumen (GO:0005788)      2.437451e-11   
1866     GTEX-UPK5-0003-SM-3NMDI_blood_male_40-49_years      2.946114e-11   
2818                                     Focal adhesion      3.253180e-11   
574       GTEX-X88G-0008-SM-47JWN_skin_male_30-39_years      3.757726e-11   
573       GTEX-P4PQ-0008-SM-48TDX_skin_male_60-69_years      3.936156e-11   
1865     GTEX-X585-0002-SM-46MVA_blood_male_50-59_years      5.051265e-11   

...

                                                   Term  Adjusted P-value  \
622    GTEX-QVUS-2826-SM-3GADB_brain_female_60-69_years          0.039768   
617   GTEX-XLM4-0011-R4B-SM-4AT5C_brain_male_60-69_y...          0.042022   
621   GTEX-QVUS-0011-R4A-SM-3GAE7_brain_female_60-69...          0.046535   
626   GTEX-RNOR-0011-R4A-SM-3GAD3_brain_female_50-59...          0.054328   
620   GTEX-T5JC-0011-R4A-SM-32PLT_brain_male_20-29_y...          0.055930   
625    GTEX-QVJO-1426-SM-2S1QY_brain_female_60-69_years          0.059833   
619   GTEX-QVJO-0011-R2A-SM-2S1QK_brain_female_60-69...          0.062010   
623   GTEX-QDT8-0011-R1A-SM-32PKS_brain_female_30-39...          0.063397   
627      GTEX-XLM4-3026-SM-4AT6L_brain_male_60-69_years          0.063748   
624   GTEX-X4XX-0011-R1B-SM-3P622_brain_male_60-69_y...          0.064916   
628   GTEX-P44H-0011-R1A-SM-3NM8J_brain_male_40-49_y...          0.066642   
618   GTEX-R55F-0011-R6A-SM-2TF4L_brain_male_50-59_y...          0.070917   

2019-12-16 16:46:57,497 Warning: No enrich terms using library GO_Molecular_Function_2018 when cutoff = 0.05
2019-12-16 16:47:00,108 Warning: No enrich terms using library GO_Biological_Process_2018 when cutoff = 0.05
2019-12-16 16:47:02,594 Warning: No enrich terms using library GO_Cellular_Component_2018 when cutoff = 0.05
2019-12-16 16:47:11,218 Warning: No enrich terms using library GTEx_Tissue_Sample_Gene_Expression_Profiles_down when cutoff = 0.05
2019-12-16 16:47:13,707 Warning: No enrich terms using library Tissue_Protein_Expression_from_Human_Proteome_Map when cutoff = 0.05
2019-12-16 16:47:18,756 Warning: No enrich terms using library NCI-60_Cancer_Cell_Lines when cutoff = 0.05


                                                  Term  Adjusted P-value  \
393            Biconcave vertebral bodies (HP:0004586)          0.015313   
390  Morphological abnormality of the middle ear (H...          0.016727   
392                 Abnormality of dentin (HP:0010299)          0.017501   
394                        Atrophic scars (HP:0001075)          0.018744   
391                    Mild short stature (HP:0003502)          0.020418   
389                   Breech presentation (HP:0001623)          0.020908   
388                   Protrusio acetabuli (HP:0003179)          0.022316   
531  GTEX-XMK1-1326-SM-4B65Q_esophagus_male_40-49_y...          0.025474   
396                             Soft skin (HP:0000977)          0.025720   
395                       Genu recurvatum (HP:0002816)          0.028292   
397                         Tibial bowing (HP:0002982)          0.029241   
398       Delayed gross motor development (HP:0002194)          0.029814   
534  GTEX-S3

2019-12-16 16:47:26,945 Warning: No enrich terms using library GO_Cellular_Component_2018 when cutoff = 0.05
2019-12-16 16:47:38,037 Warning: No enrich terms using library Tissue_Protein_Expression_from_Human_Proteome_Map when cutoff = 0.05


                                                   Term  Adjusted P-value  \
3947                   Cortisol synthesis and secretion      9.480571e-11   
3948                                   Cushing syndrome      2.395036e-08   
3949                            Ovarian steroidogenesis      1.451128e-07   
57        glucocorticoid metabolic process (GO:0008211)      4.790973e-07   
489   GTEX-U3ZM-0226-SM-4DXTA_adrenal gland_male_40-...      7.840887e-07   
56     glucocorticoid biosynthetic process (GO:0006704)      9.581945e-07   
490   GTEX-OHPK-1326-SM-3MJGN_adrenal gland_female_5...      1.801908e-06   
492   GTEX-R53T-0226-SM-48FEH_adrenal gland_female_5...      3.393327e-06   
3950                Aldosterone synthesis and secretion      3.751379e-06   
491   GTEX-S341-0126-SM-4AD64_adrenal gland_female_4...      4.314268e-06   
4007                                          SHSYSY-RA      6.815696e-06   
493   GTEX-VJYA-0526-SM-4KL1R_adrenal gland_male_60-...      1.095827e-05   

In [ ]:
topic_pvalues = []
topic_gos = []
for itopic,topic in enumerate(df_topics.columns):
    try:
        enriched_topic = pd.read_csv("%s/gsea/gsea_level_%d_topic%d.csv"%(directory,l,itopic))
        if len(enriched_topic.index) >0:
            p_val = np.sort(enriched_topic['Adjusted P-value'])[0]
            topic_pvalues.append(-np.log10(p_val))
            for goc in enriched_topic['Gene_set'][:10].unique():
                topic_gos.append(goc)
        print(topic)
    except:
        print("error", sys.exc_info()[0])

In [ ]:
fig = plt.figure()
x = np.arange(1,1+len(topic_pvalues))
c, _, _ = plt.hist(topic_pvalues, histtype='step', lw=2)
plt.plot([-np.log10(0.05) for _ in np.linspace(1,10,num=10)],np.arange(0,np.max(c)+5,(np.max(c)+5)/10), ls='--', lw=5, label="$\\alpha=0.05$")
plt.xlabel('-log(P-value)', fontsize=16)
plt.ylabel("number of topics")
#plt.ylim(0,0.055)
#plt.yscale('log')
plt.legend(fontsize=16)
fig.savefig("%s/pvaluescrosstopic(%d).png"%(directory,l))

In [ ]:
fig = plt.figure(figsize=(20,10))
gos, goscounts = np.unique(topic_gos,return_counts=True)
plt.barh(["\n".join(wrap(str(l).replace('_',' '),20)) for l in gos], goscounts)
plt.yticks(fontsize=15)
plt.show()
fig.savefig("%s/pvaluecategories(%d).pdf"%(directory,l))